In [1]:
# Import libraries
import numpy as np
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay

In [2]:
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go

import plotly
plotly.tools.set_credentials_file(username='raphaelrrcoelho', api_key='H2lY25XX3A0IkijhzRmA')

In [3]:
from modules import io, web, data, risk
import pandas_datareader.data as pd_web
import quandl

# from statsmodels import regression
# from scipy.stats import norm
# import statsmodels.api as sm

# from financial_risk import web, data, risk, io, plot, report

In [4]:
br_holidays = io.get_holidays()

In [5]:
start = pd.datetime.today().date() - CustomBusinessDay(800, holidays=br_holidays) # 252 Business Days Equals * 3 (Years)
end = pd.datetime.today().date() - CustomBusinessDay(1, holidays=br_holidays) # Yesterday

date_index = pd.date_range(start, end, freq=CustomBusinessDay(holidays=br_holidays))
print(start, end)

2015-10-16 00:00:00 2019-01-11 00:00:00


In [6]:
tickers = io.tickers(file='database.xlsx')

In [7]:
stocks = web.stocks(tickers, start, end)
stocks = stocks[:, date_index, :]

C:\Users\Raphael\Downloads\hedge-fund\modules\web.py:16: FutureWarning:


Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.


C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexing.py:1147: FutureWarning:


Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.


C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning:


Passing list-likes to .loc o

In [8]:
risk_free = web.risk_free(start, end, date_index)
cc_risk_free = np.log(1 + risk_free)

C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexing.py:116: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [9]:
close_prices = stocks.loc[:, :, 'Close'].fillna(method='ffill').fillna(method='bfill').dropna(axis=1)
adj_close_prices = stocks.loc[:, :, 'Adj Close'].fillna(method='ffill').fillna(method='bfill').dropna(axis=1)
volumes = stocks.loc[:, :, 'Volume'].fillna(method='ffill').fillna(method='bfill').dropna(axis=1)

In [10]:
adj_close_prices.head()

,ABEV3,AZUL4,BBAS3,BBDC4,BBSE3,BEEF3,BRFS3,BRML3,CCRO3,CMIG4,...,SMLS3,SUZB3,TIMP3,UGPA3,VALE3,VIVT4,VLID3,WEGE3,ibov,ibrx
2015-10-16,17.382921,22.4,14.330626,14.004691,23.823763,12.804342,65.470482,7.208724,11.570652,5.472564,...,23.733704,20.25489,7.243366,63.132771,16.851244,30.634989,37.119846,11.391292,47236.0,8079.060059
2015-10-19,17.545124,22.4,14.964412,13.985579,24.205603,12.852991,64.984077,7.525590,11.641475,5.503348,...,23.131678,20.25489,7.196575,63.634418,16.284048,30.270290,36.958462,11.327244,47447.0,8111.149902
2015-10-20,17.644249,22.4,14.480268,13.655386,23.624537,12.648666,65.849876,7.393562,11.287362,5.818931,...,23.232016,20.25489,7.177858,63.866665,16.110229,30.245979,36.958462,11.355657,47077.0,8052.120117
2015-10-21,17.752386,22.4,14.656321,13.674411,23.607933,12.580559,66.686501,7.591605,11.136864,5.895895,...,23.424973,20.25489,7.196575,64.098900,16.549351,30.537739,37.805725,11.149261,47026.0,8055.299805
2015-10-22,17.716339,22.4,14.876383,13.877637,23.707544,12.745964,67.776054,7.743431,11.296215,5.980569,...,23.154833,20.25489,7.308874,64.925690,16.933578,31.056416,37.523281,11.284485,47772.0,8177.060059


In [11]:
simple_returns = data.simple_returns(adj_close_prices)
cc_returns = np.log(1 + simple_returns)

In [12]:
volumes_diff = data.simple_returns(volumes)

In [13]:
# ewm_cov = risk.ewm_cov(cc_returns, window=75)
# ewm_mean = risk.ewm_mean(cc_returns, window=75)
# ewm_var = risk.ewm_variance(cc_returns, window=75)

In [14]:
# betas = risk.get_betas(ewm_cov).tail(1)
# betas = pd.Series(betas.ix[0, :].values, index=betas.columns, name='beta')
# tickers['beta'] = betas

In [15]:
# variance = pd.Series(ewm_var.tail(1).ix[0, :].values, index=ewm_var.tail(1).columns, name='daily_volatility')
#  tickers['daily_volatility'] = variance.head(-1) ** 0.5

In [16]:
# mean = pd.Series(ewm_mean.tail(1).ix[0, :].values, index=ewm_mean.tail(1).columns, name='average_return')

In [17]:
# tickers['daily_value_at_risk_95'] = risk.parametric_value_at_risk(mean.head(-1), variance.head(-1) ** 0.5, 0.05)
# tickers['daily_value_at_risk_99'] = risk.parametric_value_at_risk(mean.head(-1), variance.head(-1) ** 0.5, 0.01)

In [18]:
# sharpe_ratio = risk.sharpe_ratio(mean, variance)
# tickers['sharpe_ratio'] = sharpe_ratio

In [19]:
io.save_data(tickers, adj_close_prices, close_prices, simple_returns, cc_returns, file='database.xlsx')

# TOP 5 Analysis

In [98]:
top5 = pd.read_excel("portfolios.xlsx", sheet_name='TOP5')
top5['start'] = pd.to_datetime(top5['start'])

In [99]:
top5['end'] = top5['start'].shift(-1) - CustomBusinessDay(1, holidays=br_holidays)

top5 = top5.loc[:, ['start', 'end', 'ticker_1', 'ticker_2', 'ticker_3', 'ticker_4', 'ticker_5']]

top5.iloc[-1, 1] = pd.datetime(2019, 1, 10) #pd.datetime.today().date() - CustomBusinessDay(1, holidays=br_holidays) # Yesterday

C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexes\datetimes.py:962: PerformanceWarning:

Non-vectorized DateOffset being applied to Series or DatetimeIndex



In [100]:
top5 = top5.set_index(['start', 'end'])

top5.tail()

,,ticker_1,ticker_2,ticker_3,ticker_4,ticker_5
start,end,,,,,
2018-09-24,2018-10-05,ESTC3,BEEF3,RAPT4,BBDC4,PETR4
2018-10-08,2018-10-19,ESTC3,BEEF3,RAPT4,BBDC4,GGBR4
2018-10-22,2018-11-16,ESTC3,BEEF3,AZUL4,BBDC4,GGBR4
2018-11-19,2018-12-19,ESTC3,BEEF3,AZUL4,CYRE3,PETR4
2018-12-20,2019-01-10,ESTC3,BEEF3,AZUL4,FLRY3,PETR4


In [101]:
starts = top5.index.get_level_values(0)
ends = top5.index.get_level_values(1)

returns_cc = []
ibov_returns_cc = []
ibrx_returns_cc = []
risk_free_cc = []

In [102]:
for _, row in top5.iterrows():
    start = row.name[0]
    end = row.name[1]
    allocation = row.values

    returns_cc.append( cc_returns.loc[start:end, allocation].sum().sum() / 5 )
    
    ibov_returns_cc.append( cc_returns.loc[start:end, 'ibov'].sum() )
    
    ibrx_returns_cc.append( cc_returns.loc[start:end, 'ibrx'].sum() )
    
    risk_free_cc.append( cc_risk_free.loc[start:end, 'cdi'].sum() )

In [103]:
returns = np.exp(returns_cc) - 1
cum_returns = np.exp(np.cumsum(returns_cc)) - 1

ibov_returns = np.exp(ibov_returns_cc) - 1
ibov_cum_returns = np.exp(np.cumsum(ibov_returns_cc)) - 1

ibrx_returns = np.exp(ibrx_returns_cc) - 1
ibrx_cum_returns = np.exp(np.cumsum(ibrx_returns_cc)) - 1

cdi_returns = np.exp(risk_free_cc) - 1
cdi_cum_returns = np.exp(np.cumsum(risk_free_cc)) - 1

In [104]:
allocations = [ row.values for _, row in top5.iterrows() ]
allocations = [ ' '.join(i) for i in allocations ]

allocations = [
    date + ", " + allocation + ", " + "{0:.2%}".format(ret)
    for date, allocation, ret
    in zip(ends.strftime("%Y-%m-%d").tolist(), allocations, returns)
]

In [105]:
cum_allocations = [ row.values for _, row in top5.iterrows() ]
cum_allocations = [ ' '.join(i) for i in cum_allocations ]

cum_allocations = [
    date + ", " + cum_allocation + ", " + "{0:.2%}".format(cum_ret)
    for date, cum_allocation, cum_ret
    in zip(ends.strftime("%Y-%m-%d").tolist(), cum_allocations, cum_returns)
]

In [106]:
ibov_text = [
    date + ", " + "{0:.2%}".format(ret)
    for date, ret
    in zip(ends.strftime("%Y-%m-%d").tolist(), ibov_returns)
]

In [107]:
cum_ibov_text = [
    date + ", " + "{0:.2%}".format(cum_ret)
    for date, cum_ret
    in zip(ends.strftime("%Y-%m-%d").tolist(), ibov_cum_returns)
]

In [108]:
ibrx_text = [
    date + ", " + "{0:.2%}".format(ret)
    for date, ret
    in zip(ends.strftime("%Y-%m-%d").tolist(), ibrx_returns)
]

In [109]:
cum_ibrx_text = [
    date + ", " + "{0:.2%}".format(cum_ret)
    for date, cum_ret
    in zip(ends.strftime("%Y-%m-%d").tolist(), ibrx_cum_returns)
]

In [110]:
cum_cdi_text = [
    date + ", " + "{0:.2%}".format(cum_cdi)
    for date, cum_cdi
    in zip(ends.strftime("%Y-%m-%d").tolist(), cdi_cum_returns)
]

In [111]:
# Add initial points for plotting
ends = np.insert(ends, 0, starts[0])

allocations = np.insert(allocations, 0, [starts[0].strftime("%Y-%m-%d")])
cum_allocations = np.insert(cum_allocations, 0, [starts[0].strftime("%Y-%m-%d")])
returns = np.insert(returns, 0, 0)
cum_returns = np.insert(cum_returns, 0, 0)

ibov_text = np.insert(ibov_text, 0, [starts[0].strftime("%Y-%m-%d")])
cum_ibov_text = np.insert(cum_ibov_text, 0, [starts[0].strftime("%Y-%m-%d")])
ibov_returns = np.insert(ibov_returns, 0, 0)
ibov_cum_returns = np.insert(ibov_cum_returns, 0, 0)

ibrx_text = np.insert(ibrx_text, 0, [starts[0].strftime("%Y-%m-%d")])
cum_ibrx_text = np.insert(cum_ibrx_text, 0, [starts[0].strftime("%Y-%m-%d")])
ibrx_returns = np.insert(ibrx_returns, 0, 0)
ibrx_cum_returns = np.insert(ibrx_cum_returns, 0, 0)

cum_cdi_text = np.insert(cum_cdi_text, 0, [starts[0].strftime("%Y-%m-%d")])
cdi_returns = np.insert(cdi_returns, 0, 0)
cdi_cum_returns = np.insert(cdi_cum_returns, 0, 0)

In [112]:
data = [
    go.Scatter(
        x = ends,
        y = returns,
        text = allocations,
        hoverinfo = 'text',
        marker = dict(
            color = 'green'
        ),
        name='Top 5 Daily Returns'
    ),
    
    go.Scatter(
        x = ends,
        y = ibov_returns,
        text = ibov_text,
        hoverinfo = 'text',
        marker = dict(
            color = 'blue'
        ),
        name='Ibov Daily Returns'
    )
]

py.iplot(data, filename = "top5_returns")

In [113]:
data = [
    go.Scatter(
        x = ends,
        y = cum_returns,
        text = cum_allocations,
        hoverinfo = 'text',
        marker = dict(
            color = 'green'
        ),
        name='Top 5 Cumulative Returns'
    ),
    
    go.Scatter(
        x = ends,
        y = ibov_cum_returns,
        text = cum_ibov_text,
        hoverinfo = 'text',
        marker = dict(
            color = 'blue'
        ),
        name='Ibov Cumulative Returns'
    ),
    go.Scatter(
        x = ends,
        y = ibrx_cum_returns,
        text = cum_ibrx_text,
        hoverinfo = 'text',
        marker = dict(
            color = 'yellow'
        ),
        name='Ibrx Cumulative Returns'
    ),
    go.Scatter(
        x = ends,
        y = cdi_cum_returns,
        text = cum_cdi_text,
        hoverinfo = 'text',
        marker = dict(
            color = 'red'
        ),
        name='CDI Cumulative Returns'
    )
]

layout = dict(
    title = "Cumulative Returns",
    yaxis = dict(
        tickformat= '.1%'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = "top5_cumulative_returns")

#### Cumulative returns

In [157]:
# TOP5
top5_cum_return = cum_returns.item(-1)
"Top 5 cumulative return {0:.2%}".format(top5_cum_return)

'Top 5 cumulative return 285.58%'

In [156]:
# IBOV
ibov_cum_return = ibov_cum_returns.item(-1)
"IBOV cumulative return {0:.2%}".format(ibov_cum_return)

'IBOV cumulative return 114.89%'

In [155]:
# IBRX
ibrx_cum_return = ibrx_cum_returns.item(-1)
"IBRX cumulative return {0:.2%}".format(ibrx_cum_return)

'IBRX cumulative return 110.06%'

In [154]:
# CDI
cdi_cum_return = cdi_cum_returns.item(-1)
"CDI cumulative return {0:.2%}".format(cdi_cum_return)

'CDI cumulative return 33.32%'

#### Annualized returns

In [118]:
n_days = cc_returns.shape[0]
n_days

799

In [153]:
# TOP5
annualized_top5_return = (1 + cum_returns.item(-1))** ( 252. / n_days) - 1
"Annualized Top 5 {0:.2%}".format(annualized_top5_return)

'Annualized Top 5 53.06%'

In [152]:
# IBOV
annualized_ibov_return = (1 + ibov_cum_returns.item(-1))** ( 252. / n_days) - 1
"Annualized IBOV {0:.2%}".format(annualized_ibov_return)

'Annualized IBOV 27.28%'

In [151]:
# IBRX
annualized_ibrx_return = (1 + ibrx_cum_returns.item(-1))** ( 252. / n_days) - 1
"Annualized IBRX {0:.2%}".format(annualized_ibrx_return)

'Annualized IBRX 26.38%'

In [148]:
# CDI
annualized_cdi_return = (1 + cdi_cum_returns.item(-1))** ( 252. / n_days) - 1
"Annualized CDI {0:.2%}".format(annualized_cdi_return)

'Annualized CDI 9.49%'

#### Risk Metrics

In [123]:
start = top5.index.get_level_values(0)[0]
end = top5.index.get_level_values(1)[-1]

date_index = pd.date_range(start, end, freq=CustomBusinessDay(holidays=br_holidays))
print(start, end)

2015-12-30 00:00:00 2019-01-10 00:00:00


In [125]:
period_cdi = risk_free.loc[start:end, 'cdi']

In [126]:
period_ibov = cc_returns.loc[start:end, 'ibov']

In [127]:
top5_daily_returns = []

for _, row in top5.iterrows():
    start = row.name[0]
    end = row.name[1]
    allocation = row.values
    
    current_date = start
    
    while current_date <= end:
        top5_daily_returns.append( cc_returns.loc[current_date, allocation].sum() / 5 )
        
        current_date = current_date + CustomBusinessDay(1, holidays=br_holidays)

In [128]:
top5_daily_returns = pd.Series(np.exp(top5_daily_returns) - 1)
period_ibov = np.exp(period_ibov) - 1

In [129]:
d = {
    'top5': top5_daily_returns.values.tolist(),
    'cdi': period_cdi.values.tolist(),
    'ibov': period_ibov.values.tolist()
}
returns = pd.DataFrame(data = d, index=date_index)

In [130]:
# Covariance
top5_ibov_cov = returns.cov().loc['top5', 'ibov']
top5_ibov_cov

0.0002047731528848022

In [147]:
# Correlation
top5_ibov_corr = returns.corr().loc['top5', 'ibov']
print('Top5 x Ibov Correl:', top5_ibov_corr)

Top5 x Ibov Correl: 0.829107438541811


In [132]:
# Variance
print("TOP5 Annualized Var: {}".format(returns['top5'].var() * 252))
print("IBOV Annualized Var: {}".format(returns['ibov'].var() * 252))

TOP5 Annualized Var: 0.07581229738227352
IBOV Annualized Var: 0.051095874362583955


In [133]:
# Annualized Std Deviation
print("TOP5 Annualized Std Dev: {0:.3%}".format(returns['top5'].std() * np.sqrt(252)))
print("IBOV Annualized Std Dev: {0:.3%}".format(returns['ibov'].std() * np.sqrt(252)))

TOP5 Annualized Std Dev: 27.534%
IBOV Annualized Std Dev: 22.604%


In [134]:
# Tracking Error
top5_ibov_tracking_error =  (returns['top5'] - returns['ibov']).std()
print("TOP5 Annualized Tracking Error: {0:.3%}".format(top5_ibov_tracking_error * np.sqrt(252)))

TOP5 Annualized Tracking Error: 15.396%


In [144]:
# Beta
top5_ibov_beta = top5_ibov_cov / returns['ibov'].var()
print('Beta:', top5_ibov_beta)

Beta: 1.009921743598881


In [145]:
# Sharpe
sharpe_ratio = (annualized_top5_return - annualized_cdi_return) / (returns['top5'].std() * np.sqrt(252))
print('Sharpe Ratio:', sharpe_ratio)

Sharpe Ratio: 1.5821695323086031


In [146]:
# Information Ratio
print('Information Ratio:', (annualized_top5_return - annualized_ibov_return) / (top5_ibov_tracking_error * np.sqrt(252)))

Information Ratio: 1.6740942279261386


### Itaú's Suggestion Effect

In [138]:
windows = pd.DataFrame(columns=['mean_returns', 'skew_returns', 'kurtosis_returns', 'mean_volume_diff',  'skew_volume_diff'], index=range(0, 6))

In [139]:
for i in range(0, 6):
    initial_allocation = top5.iloc[0].values
    
    dates = []
    new_stocks = []
    new_returns = []
    new_volumes = []

    for _, row in top5.iterrows():
        start = row.name[0]
        end = row.name[1]
        allocation = row.values

        new_stock = np.setdiff1d(initial_allocation, allocation)

        if new_stock.size > 0:
            ns = np.setdiff1d(initial_allocation, allocation)[0]

            dates.append(start + CustomBusinessDay(i, holidays=br_holidays))
            new_stocks.append(ns)

            new_returns.append( np.exp(cc_returns.loc[start + CustomBusinessDay(i, holidays=br_holidays), ns]) - 1 )

            new_volumes.append( 
                volumes.loc[start + CustomBusinessDay(i, holidays=br_holidays), ns] / volumes.loc[start - CustomBusinessDay(1, holidays=br_holidays), ns]  - 1
            )

        initial_allocation = allocation
    
    d = { 'tickers': new_stocks, 'returns': new_returns, 'volumes_diff': new_volumes }
    insertions = pd.DataFrame(data = d, index=dates)
    
    windows.loc[i , 'mean_returns'] = insertions.returns.mean()
    windows.loc[i , 'skew_returns'] = insertions.returns.skew()
    windows.loc[i , 'kurtosis_returns'] = insertions.returns.kurt()
    windows.loc[i , 'mean_volume_diff'] = insertions.volumes_diff.mean()
    windows.loc[i , 'skew_volume_diff'] = insertions.volumes_diff.skew()
    #windows.loc[i , 'returns'] = insertions.returns.values

In [140]:
data = [
        go.Scatter(
        x = list(range(0, 6)),
        y = windows.mean_returns
    )
]

layout = go.Layout(
    title='Average Returns D+i After Itaú Suggestion',
    xaxis=dict(title='D + i'),
    yaxis=dict(
        title='Average Return',
        tickformat= '.1%'
    )
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='windows_mean_returns')

In [141]:
data = [
    go.Scatter(
        x = list(range(0, 6)),
        y = windows.skew_returns
    )
]

layout = go.Layout(
    title='Skewness of Returns D+i After Itaú Suggestion',
    xaxis=dict(title='D + i'),
    yaxis=dict(title='Skewness of Return')
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='windows_skew_returns')

In [142]:
data = [
    go.Scatter(
        x = list(range(0, 6)),
        y = windows.mean_volume_diff
    )
]

layout = go.Layout(
    title='Average of Volume Diff D+i After Itaú Suggestion',
    xaxis=dict(title='D + i'),
    yaxis=dict(
        title='Average Volume Diff',
        tickformat= '.1%'
    )
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='windows_mean_volume_diff')

In [143]:
data = [
    go.Scatter(
        x = list(range(0, 6)),
        y = windows.skew_volume_diff
    )
]

layout = go.Layout(
    title='Skewness of Volume Diff D+i After Itaú Suggestion',
    xaxis=dict(title='D + i'),
    yaxis=dict(title='Skewness of Volume Diff')
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='windows_skew_volume_diff')